### Canned Estimators

In this notebook we'll demonstrate how to use two Canned Estimators (these encapsulate the lower-level TensorFlow code we've seen so far, and use an API loosely inspired by [scikit-learn](scikit-learn.org). There are several advantages to Canned Estimators.

* If you're using Estimators, you won't have to manage Sessions, or write your own logic for TensorBoard, or for saving and loading checkpoints.


* You'll get out-of-the-box distributed training (of course, you will have to take care to read your data efficiently, and set up a cluster).

Here, we'll read data using [input functions](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn), which are appropriate for in-memory data. 

* These provide batching and other features for you, so you don't have to write that code yourself.


* In the strucuted data notebook, we'll use the new [Dataset API](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/docs_src/programmers_guide/datasets.md) - which is a popular abstraction, and a great way to efficiently read and pre-process large datasets efficiently.

Although the Estimators we'll use here are relative simple (a LinearClassifier, and a Fully Connected Deep Neural Network), we also provide more interesting ones (including for [TensorFlow Wide and Deep](https://www.tensorflow.org/tutorials/wide_and_deep). I'm also excited that additional Estimators are on their way - stay tuned in the upcoming months.

Also note that Estimators log quite a lot of output

In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [15]:
# We'll use Keras (included with TensorFlow) to import the data
(x_train, y_train), (x_test, y_test) = tf.contrib.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# Normalize the color values to 0-1
# (as imported, they're 0-255)
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 14s 1us/step
60000 train samples
10000 test samples


Here's our input function. 

* By setting ```num_epochs``` to ```None```, we'll loop over the data indefinitely so we can train for as long as we like.
* The default ```batch_size``` is ```128```, but you can provide a different parameter if you like.

You can read more about the numpy input function [here](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn). We also provide a nice one for [Pandas](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn), more on that later.

In [16]:
train_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train},
    y_train, 
    num_epochs=None, # repeat forever
    shuffle=True # 
)

In [17]:
test_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test},
    y_test,
    num_epochs=1, # loop through the dataset once
    shuffle=False # don't shuffle the test data
)

In [18]:
# define the features for our model
# the names must match the input function
feature_spec = [tf.feature_column.numeric_column('x', shape=784)]

Here, we'll create a ```LinearClassifier``` - this is identical to our Softmax (aka, multiclass logistic regression model) from the second notebok.

In [19]:
estimator = tf.estimator.LinearClassifier(feature_spec, 
                                          n_classes=10,
                                          model_dir="./graphs/canned/linear")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/linear', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028E1C443390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# I've arbitrarily decided to train for 1000 steps
estimator.train(train_input, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/canned/linear\model.ckpt.
INFO:tensorflow:loss = 294.73093, step = 1
INFO:tensorflow:global_step/sec: 481.954
INFO:tensorflow:loss = 42.561043, step = 101 (0.207 sec)
INFO:tensorflow:global_step/sec: 528.063
INFO:tensorflow:loss = 42.265114, step = 201 (0.189 sec)
INFO:tensorflow:global_step/sec: 545.464
INFO:tensorflow:loss = 35.247963, step = 301 (0.183 sec)
INFO:tensorflow:global_step/sec: 510.034
INFO:tensorflow:loss = 45.110916, step = 401 (0.196 sec)
INFO:tensorflow:global_step/sec: 539.861
INFO:tensorflow:loss = 40.501846, step = 501 (0.185 sec)
INFO:tensorflow:global_step/sec: 597.987
INFO:tensorflow:loss = 34.06955, step = 601 (0.167 sec)
INFO:tensorflow:global_step/sec: 548.178
INFO:tensor

In [21]:
# We should see about 90% accuracy here.
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-03-18:51:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-03-18:51:12
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9245, average_loss = 0.27385157, global_step = 1000, loss = 34.664757
{'accuracy': 0.9245, 'average_loss': 0.27385157, 'loss': 34.664757, 'global_step': 1000}


Here's how you would print individual predictions.

In [22]:
MAX_TO_PRINT = 5

# This returns a generator object
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Example 0. True: 7, Predicted: 7
Example 1. True: 2, Predicted: 2
Example 2. True: 1, Predicted: 1
Example 3. True: 0, Predicted: 0
Example 4. True: 4, Predicted: 4


Here's how easy it is to switch the model to a fully connected DNN.

In [23]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[256], # we will arbitrarily use two layers
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/canned/deep")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/deep', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028E1E9940B8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
# I've arbitrarily decided to train for 2000 steps
estimator.train(train_input, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./graphs/canned/deep\model.ckpt.
INFO:tensorflow:loss = 300.9713, step = 1
INFO:tensorflow:global_step/sec: 241.222
INFO:tensorflow:loss = 45.72307, step = 101 (0.421 sec)
INFO:tensorflow:global_step/sec: 203.706
INFO:tensorflow:loss = 30.810621, step = 201 (0.485 sec)
INFO:tensorflow:global_step/sec: 277.421
INFO:tensorflow:loss = 16.742062, step = 301 (0.360 sec)
INFO:tensorflow:global_step/sec: 207.531
INFO:tensorflow:loss = 19.270012, step = 401 (0.485 sec)
INFO:tensorflow:global_step/sec: 284.562
INFO:tensorflow:loss = 7.9361863, step = 501 (0.364 sec)
INFO:tensorflow:global_step/sec: 201.341
INFO:tensorflow:loss = 14.301344, step = 601 (0.481 sec)
INFO:tensorflow:global_step/sec: 208.758
INFO:tensorflo

In [25]:
# Expect accuracy around 97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-03-18:51:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-03-18:51:23
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9758, average_loss = 0.08121418, global_step = 2000, loss = 10.280275
{'accuracy': 0.9758, 'average_loss': 0.08121418, 'loss': 10.280275, 'global_step': 2000}


If you like, you can compare these runs with TensorBoard.

``` $ tensorboard --logdir=graphs/canned/ ```